In [1]:
!pip install xlrd
!pip install openpyxl

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import re
import os
from glob import glob
import time
from tqdm import tqdm
from packages import utils

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

## 데이터 불러오기

In [3]:
now_path = os.getcwd()
data_path = utils.recurrent_find_data_path(now_path)
bucket_files = utils.get_data_paths(data_path)
file_dict = utils.files_to_pd_dict(bucket_files)

100%|██████████| 19/19 [00:22<00:00,  1.19s/it]


In [4]:
keys = list(file_dict.keys())
print(f"불러온 데이터 파일명들 : {keys}")

불러온 데이터 파일명들 : ['sample_answersheet', 'test', 'train', '교통_최종본(0416)', '여권 최종본(0416)', 'G 숙박업(7,113)_new', 'F 카페(7,859)_new', 'I 부동산(8,131)_new', 'D 소매점(14,949)_new', 'A 음식점(15,726)_new', '상수도_최종본(0416)', 'H 관광여가오락(4,949)_new', 'C 학원(4,773)_new', '차량등록_최종본(0429)', 'E 생활서비스(11,087)_new', 'B 의류(15,826)_new']


TUNiB 데이터 : sample_answersheet, test, train  
AI_HUB 데이터 : 나머지 데이터

In [72]:
train = file_dict["train"][["class", "conversation"]]
test = file_dict["test"].T.reset_index()
test_answer = file_dict["sample_answersheet"]

base = pd.DataFrame(file_dict[keys[-1]]["SENTENCE"])
base["class"] = "일반 대화"
base.columns = ["conversation", "class"]

train_data = pd.concat([base, train])
train_data = train_data.reset_index(drop=True)

## 데이터 전처리

In [80]:
def prep(x):
    x = re.sub("[^ㄱ-ㅎ가-힣]+", " ", x)
    x = re.sub("[ ]+", " ", x)
    x = x.strip()
    return x

prep(ex)

'준하야 넌 대가리가 왜이렇게 크냐 내 머리가 뭐 밥먹으면 대가리만 크냐 너는 아니 무슨 머리만 큰다고 그러는거야 주위를 둘러봐라 내가 살면서 너보다 큰 대가리를 못봤어 나한테 왜그러는거야 무식하게 대가리만 크면 뭐하냐 멍청해가지고는 쯧쯧 넌 내가 그렇게 싫니 당연하지 언제 이 세상에서 없어지냐 너 내가 그날만 기다리고 있어 조만간 사라져줄게 너 눈앞에서 그거 요즘들은 소식중에 제일 좋은 소식이다'

In [82]:
train_data["pre_conversation"] = train_data["conversation"].apply(lambda x : prep(x))

In [85]:
from konlpy.tag import Mecab

In [88]:
mecab = Mecab()

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

In [84]:
train_data

,conversation,class,pre_conversation
0,신발은 여기 있는 게 다예요?,일반 대화,신발은 여기 있는 게 다예요
1,네 성인이나 아동 다 있어요,일반 대화,네 성인이나 아동 다 있어요
2,발 사이즈 몇 신으세요?,일반 대화,발 사이즈 몇 신으세요
3,230이요,일반 대화,이요
4,편하게 신을 수 있는 거 찾으세요?,일반 대화,편하게 신을 수 있는 거 찾으세요
...,...,...,...
19771,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐 내 머리가 뭐 밥먹으면 대가리만 크냐 너는 아니 ...
19772,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...,갈취 대화,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 억만 보내 예 선생님 ...
19773,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지 왜 그래 들리겠어 들리라...
19774,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...,갈취 대화,누구맘대로 여기서 장사하래 이게 무슨일입니까 남의 구역에서 장사하려면 자릿세를 내야...
